<a href="https://colab.research.google.com/github/EmirhanEge1/Wikipedia-Speedrunner/blob/main/Speedrun_Wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U sentence-transformers wikipedia-api

In [ ]:
import wikipediaapi
import time
import heapq
import torch
import pandas as pd
import random
import re
from sentence_transformers import SentenceTransformer, util

device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer('all-MiniLM-L6-v2').to(device)
wiki_en = wikipediaapi.Wikipedia(
    user_agent='DataEngineerBot/1.0 (email@example.com)',
    language='en',
    extract_format=wikipediaapi.ExtractFormat.WIKI
)




In [ ]:
def is_valid_link(link):
    exclude = ["Category:", "File:", "Help:", "Template:", "Talk:", "Portal:", "Special:", "Wikipedia:"]
    return not any(link.startswith(prefix) for prefix in exclude)

def fast_pre_filter(links, target_page, top_n=50):
    target_words = set(re.findall(r'\w+', target_page.lower()))
    scored_links = []

    for link in links:
        score = 0
        link_lower = link.lower()
        for word in target_words:
            if word in link_lower:
                score += 1
        scored_links.append((score, link))

    scored_links.sort(key=lambda x: x[0], reverse=True)
    return [link for score, link in scored_links[:top_n]]

In [ ]:
def start_wikipedia_speedrunner(start_page, target_page, timeout=60):
    start_time = time.perf_counter()
    target_emb = model.encode(target_page, convert_to_tensor=True, normalize_embeddings=True)
    target_keywords = [w.lower() for w in target_page.split() if len(w) > 2]

    queue = [(-1.0, start_page, [start_page])]
    visited = {start_page.lower()}
    step_count = 0

    while queue:
        elapsed = time.perf_counter() - start_time
        if elapsed > timeout:
            return {"steps": 0, "explored": step_count, "success": False, "status": "Timed Out", "duration": elapsed, "path": []}

        neg_score, current_title, path = heapq.heappop(queue)
        step_count += 1

        if current_title.lower() == target_page.lower():
            return {"steps": len(path)-1, "explored": step_count, "success": True, "status": "Success", "duration": elapsed, "path": path}

        try:
            time.sleep(0.04)
            page = wiki_en.page(current_title)
            if not page.exists(): continue
            all_links = list(page.links.keys())
        except Exception as e:
            time.sleep(1.0)
            continue

        raw_links = [l for l in all_links if l.lower() not in visited and is_valid_link(l)]
        turbo_links = fast_pre_filter(raw_links, target_page, top_n=50)

        if not turbo_links: continue

        try:
            link_embs = model.encode(turbo_links, convert_to_tensor=True, normalize_embeddings=True, batch_size=128)
            cos_scores = util.dot_score(target_emb, link_embs)[0]
        except: continue

        for i, link_name in enumerate(turbo_links):
            link_score = cos_scores[i].item()
            link_lower = link_name.lower()

            for kw in target_keywords:
                if kw in link_lower: link_score += 0.5



            if i < 15: link_score += 0.05

            link_score -= (len(path) * 0.02)

            if link_lower not in visited:
                visited.add(link_lower)
                heapq.heappush(queue, (-link_score, link_name, path + [link_name]))

    return {"steps": 0, "explored": step_count, "success": False, "status": "No Path", "duration": time.perf_counter() - start_time, "path": []}

In [ ]:
start_wikipedia_speedrunner( "Mushroom" ,"Walmart")

In [ ]:
"""
Modes:
- 'vital': Targets the top 1,000/10,000 essential articles.
- 'academic': Focuses on academic disciplines and their specialized branches.
- 'deep': Scrapes links from the deeper layers of a random topic.
"""

def run_benchmark(num_tests=50, mode="vital"):
    print(f"\nMode='{mode}'...")
    pool = []

    if mode == "vital":
        page = wiki_en.page("Wikipedia:Vital articles")
        pool = [l for l in page.links.keys() if is_valid_link(l)]
    elif mode == "academic":
        hubs = ["Outline of academic disciplines", "List of academic fields"]
        for hub in hubs:
            p = wiki_en.page(hub)
            pool.extend([l for l in p.links.keys() if is_valid_link(l)])
        pool = list(set(pool))
    elif mode == "deep":
        base_topics = [
    "Science", "History", "Technology", "Arts",
    "Geography", "Philosophy", "Religion", "Mathematics",
    "Biology", "Medicine", "Economics", "Law",
    "Literature", "Music", "Politics", "Environment",
    "Psychology", "Sociology", "Sports", "Engineering"
]
        chosen = wiki_en.page(random.choice(base_topics))
        first_tier = random.sample(list(chosen.links.keys()), min(30, len(chosen.links)))
        for link in first_tier:
            try:
                p = wiki_en.page(link)
                sub_links = list(p.links.keys())
                pool.extend(random.sample(sub_links, min(10, len(sub_links))))
            except: continue
        pool = [l for l in pool if is_valid_link(l)]

    results = []
    print(f" {mode.upper()} Benchmark: {num_tests} ")
    print("=" * 110)

    for i in range(num_tests):
        try:
            start, end = random.sample(pool, 2)
        except ValueError: break

        print(f"[{i+1:03d}/{num_tests}] {start[:25]:<25} ➜ {end[:25]:<25}", end=" | ", flush=True)

        res = start_wikipedia_speedrunner(start, end, timeout=60)

        status = res.get("status", "Error")
        duration = round(res.get("duration", 0), 2)
        steps = res.get("steps", 0)

        current_res = {
            "Start": start, "End": end,
            "Steps": steps, "Explored": res.get("explored", 0),
            "Duration": duration, "Status": status,
            "Path": " ➜ ".join(res["path"]) if res.get("success") else ""
        }
        results.append(current_res)

        icon = "✅" if res.get("success") else "❌"
        print(f"{icon} {status:<10} | {duration:>5.2f}s | Adım: {steps}")

        if res.get("success"):
            print(f"Route: {current_res['Path']}")

        if (i + 1) % 10 == 0:
            time.sleep(3)
            pd.DataFrame(results).to_csv(f"benchmark_{mode}_interim.csv", index=False)

    df = pd.DataFrame(results)
    df.to_csv(f"benchmark_{mode}_final.csv", index=False)

    if not df.empty:
        success_rate = (len(df[df.Status == "Success"]) / len(df)) * 100
        print("=" * 110)
        print(f"📊 BAŞARI ORANI: %{success_rate:.2f} | ORT. SÜRE: {df['Duration'].mean():.2f}s")

    return df

In [ ]:
df_vital = run_benchmark(num_tests=300, mode="vital")

In [ ]:
df_academic = run_benchmark(num_tests=300, mode="academic")

In [ ]:
df_vital = run_benchmark(num_tests=300, mode="deep")